In [3]:
import configparser
import boto3
import json
import time
import pandas as pd


# Configuration from cluster.cfg file
config = configparser.ConfigParser()
config.read('cluster.cfg')



# AWS Redshift and IAM configuration
DWH_CLUSTER_TYPE = config.get("DWH", "DWH_CLUSTER_TYPE")
DWH_NUM_NODES = config.get("DWH", "DWH_NUM_NODES")
DWH_NODE_TYPE = config.get("DWH", "DWH_NODE_TYPE")
DWH_CLUSTER_IDENTIFIER = config.get("DWH", "DWH_CLUSTER_IDENTIFIER")
DWH_DB = config.get("DWH", "DWH_DB")
DWH_DB_USER = config.get("DWH", "DWH_DB_USER")
DWH_DB_PASSWORD = config.get("DWH", "DWH_DB_PASSWORD")
DWH_PORT = config.get("DWH", "DWH_PORT")
DWH_IAM_ROLE_NAME = config.get("DWH", "DWH_IAM_ROLE_NAME")



# AWS Boto3 clients
iam = boto3.client('iam', aws_access_key_id=config.get("AWS", "KEY"),
                    aws_secret_access_key=config.get("AWS", "SECRET"),
                    region_name='us-west-2')

redshift = boto3.client('redshift', aws_access_key_id=config.get("AWS", "KEY"),
                         aws_secret_access_key=config.get("AWS", "SECRET"),
                         region_name='us-west-2')

ec2 = boto3.resource('ec2', aws_access_key_id=config.get("AWS", "KEY"),
                      aws_secret_access_key=config.get("AWS", "SECRET"),
                      region_name='us-west-2')




def create_iam_role():
    
    """Create IAM role and attach policies."""
    
    try:
        print("IAM Role: Creation")
        dwhRole = iam.create_role(
            Path='/',
            RoleName=DWH_IAM_ROLE_NAME,
            Description="Allows Redshift clusters to call AWS services on your behalf.",
            AssumeRolePolicyDocument=json.dumps(
                {'Statement': [{'Action': 'sts:AssumeRole',
                                'Effect': 'Allow',
                                'Principal': {'Service': 'redshift.amazonaws.com'}}],
                 'Version': '2012-10-17'})
        )
        print("Attaching Policy")
        iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                               PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
        roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
        print(f"IAM Role Created: {roleArn} ")
        return roleArn
    
    except Exception as e:
        print(f"Error: {e}")

        
        
        
def create_redshift_cluster(roleArn):
    
    """Create a Redshift cluster"""
    
    try:
        response = redshift.create_cluster(
            ClusterType=DWH_CLUSTER_TYPE,
            NodeType=DWH_NODE_TYPE,
            NumberOfNodes=int(DWH_NUM_NODES),
            DBName=DWH_DB,
            ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
            MasterUsername=DWH_DB_USER,
            MasterUserPassword=DWH_DB_PASSWORD,
            IamRoles=[roleArn]
        )
        
        print("Cluster: Creation")
        
        return response['Cluster']['ClusterStatus']
    
    except Exception as e:
        print(f"Error: {e}")

def update_config_file(roleArn, clusterEndpoint):
    
    """Update the dwh.cfg file with the new ARN and endpoint."""
    
    config.read('dwh.cfg')
    config.set('CLUSTER', 'HOST', clusterEndpoint)
    config.set('IAM_ROLE', 'ARN', roleArn)
    with open('dwh.cfg', 'w') as configfile:
        config.write(configfile)
        
    print("dwh.cfg updated with new ARN and endpoint.")

def main():
    roleArn = create_iam_role()
    
    if roleArn:
        status = create_redshift_cluster(roleArn)
        
        if status == 'available': 
            cluster_props = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
            clusterEndpoint = cluster_props['Endpoint']['Address']
            update_config_file(roleArn, clusterEndpoint)
            print(f"Cluster creation - Status: {status}")

if __name__ == "__main__":
    main()


IAM Role: Creation
Attaching Policy
IAM Role Created: arn:aws:iam::891377030922:role/project-2-redshift-tbs89 
Cluster: Creation
